In [17]:
import pandas as pd
import numpy as np
import acquire as a
import prepare as p
import model_functions as m

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
import scikitplot as skplt



import warnings
warnings.filterwarnings('ignore')


In [18]:
#acquiring data, prepping, splitting, isolating target variable

df = a.get_heart()
df = p.rename_cols(df)
df = p.replace_cat_values(df)
df = m.df_classification_ready(df, 'sex_female')
train, val, test = p.split_data(df, 'high_risk_of_mi')
X_Train, y_Train, X_val, y_val, X_test, y_test = m.isolate_target(train, val, test, 'high_risk_of_mi')

#showing results of decision tree models
dec_tree_results = m.get_decision_tree_multiple(X_Train, y_Train, X_val, y_val)
dec_tree_results

,min_samples_per_leaf,max_depth,train_accuracy,validate_accuracy,difference
0,1,2,0.791045,0.862745,-0.071700
27,4,5,0.875622,0.862745,0.012877
24,4,2,0.791045,0.862745,-0.071700
40,6,2,0.791045,0.862745,-0.071700
50,7,4,0.850746,0.862745,-0.011999
51,7,5,0.850746,0.862745,-0.011999
19,3,5,0.905473,0.862745,0.042728
52,7,6,0.850746,0.862745,-0.011999
53,7,7,0.850746,0.862745,-0.011999
16,3,2,0.791045,0.862745,-0.071700


In [19]:
# function from modeling_functions.py copy/pasted here for editing
#Need to get y_pred_train


def get_decision_tree_multiple(X_Train, y_Train, X_val, y_val):
    metrics = []

    for j in range (1, 10):
        for i in range(2, 10):
            clf = DecisionTreeClassifier(max_depth=i, min_samples_leaf=j, random_state=123)

            clf = clf.fit(X_Train, y_Train)
            in_sample_accuracy = clf.score(X_Train, y_Train)
            out_of_sample_accuracy = clf.score(X_val, y_val)
            # make prediction on train obeservations
            y_pred = clf.predict(X_Train)
            # estimate probability
            y_pred_proba = clf.predict_proba(X_Train)

            output = {
                "min_samples_per_leaf": j,
                "max_depth": i,
                "train_accuracy": in_sample_accuracy,
                "validate_accuracy": out_of_sample_accuracy
            }
    
            metrics.append(output)

    df1 = pd.DataFrame(metrics)
    df1["difference"] = df1.train_accuracy - df1.validate_accuracy
    df1_sorted = df1.sort_values(by=['validate_accuracy'], ascending=False).head(10)

    return df1_sorted, y_pred, y_pred_proba


In [20]:
dec_tree_results, y_pred, y_pred_proba = get_decision_tree_multiple(X_Train, y_Train, X_val, y_val)

In [22]:
y_pred_proba

array([[0.6       , 0.4       ],
       [1.        , 0.        ],
       [0.4375    , 0.5625    ],
       [0.        , 1.        ],
       [0.82352941, 0.17647059],
       [0.        , 1.        ],
       [0.6       , 0.4       ],
       [0.4375    , 0.5625    ],
       [0.        , 1.        ],
       [0.69230769, 0.30769231],
       [0.4375    , 0.5625    ],
       [0.82352941, 0.17647059],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.4375    , 0.5625    ],
       [0.82352941, 0.17647059],
       [0.82352941, 0.17647059],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.6       , 0.4       ],
       [0.6       , 0.4       ],
       [1.        , 0.        ],
       [0.75      , 0.25      ],
       [1.        , 0.        ],
       [0.26666667, 0.73333333],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.75      , 0.25      ],
       [0.

In [ ]:
# confusion matrix
confusion_matrix(y_Train, y_pred)


In [ ]:
# skplt.metrics.plot_confusion_matrix(y_Train, y_pred, normalize=False, title = 'Confusion Matrix for Decision Tree')

In [ ]:
#get mode baseline 
y_Train.value_counts()


In [ ]:
y_Train

In [ ]:
y_Train['baseline_prediction'] = 1


In [ ]:
model_accuracy = (y_pred == y_Train).mean()
baseline_accuracy = (y_Train['baselibe_predicition'] == y_Train).mean()

print(f'   model accuracy: {model_accuracy:.2%}')
print(f'baseline accuracy: {baseline_accuracy:.2%}')
